In [1]:
# -----------------------------------------------
# Import required libraries
# -----------------------------------------------
import yfinance as yf  # Used to pull financial data from Yahoo Finance

# Import all financial metric functions from your custom module
from key_ratios_and_financial_metrics import (
    pe_ratio, pb_ratio, ps_ratio, ev_ebitda, ev_to_revenue, 
    gross_margin, operating_margin, net_margin,
    roa, asset_turnover, inventory_turnover, 
    dividend_yield, earnings_yield, price_to_cash_flow, 
)

# -----------------------------------------------
# Download data for RWC.AX (Reliance Worldwide Corporation)
# -----------------------------------------------
ticker = yf.Ticker("RWC.AX")

# Extract different types of financial data
info = ticker.info            # General info like market cap, price, dividend
bs = ticker.balance_sheet     # Balance sheet
fs = ticker.financials        # Income statement
cf = ticker.cashflow          # Cash flow statement

# -----------------------------------------------
# Extract key data from Yahoo Finance 'info' dictionary
# -----------------------------------------------
price = info.get("currentPrice")
eps = info.get("trailingEps")
book_value = info.get("bookValue")
dividend = info.get("dividendRate")
shares = info.get("sharesOutstanding")
ebitda = info.get("ebitda")
market_cap = info.get("marketCap")

# -----------------------------------------------
# Extract values from income statement (fs)
# -----------------------------------------------
revenue = fs.loc["Total Revenue"].iloc[0] if "Total Revenue" in fs.index else None
gross_profit = fs.loc["Gross Profit"].iloc[0] if "Gross Profit" in fs.index else None
operating_income = fs.loc["Operating Income"].iloc[0] if "Operating Income" in fs.index else None
net_income = fs.loc["Net Income"].iloc[0] if "Net Income" in fs.index else None
ebit = fs.loc["Ebit"].iloc[0] if "Ebit" in fs.index else None
interest_expense = fs.loc["Interest Expense"].iloc[0] if "Interest Expense" in fs.index else None

# -----------------------------------------------
# Extract values from balance sheet (bs)
# -----------------------------------------------
total_assets = bs.loc["Total Assets"].iloc[0] if "Total Assets" in bs.index else None
equity = bs.loc["Total Stockholder Equity"].iloc[0] if "Total Stockholder Equity" in bs.index else None
current_assets = bs.loc["Total Current Assets"].iloc[0] if "Total Current Assets" in bs.index else None
current_liabilities = bs.loc["Total Current Liabilities"].iloc[0] if "Total Current Liabilities" in bs.index else None
inventory = bs.loc["Inventory"].iloc[0] if "Inventory" in bs.index else 0
receivables = bs.loc["Net Receivables"].iloc[0] if "Net Receivables" in bs.index else None
cash = bs.loc["Cash"].iloc[0] if "Cash" in bs.index else 0

# Calculate total debt from short- and long-term components
total_debt = 0
if "Long Term Debt" in bs.index:
    total_debt += bs.loc["Long Term Debt"].iloc[0]
if "Short Long Term Debt" in bs.index:
    total_debt += bs.loc["Short Long Term Debt"].iloc[0]

# -----------------------------------------------
# Derived metrics
# -----------------------------------------------

# Capital employed = total assets - current liabilities
capital_employed = total_assets - current_liabilities if total_assets and current_liabilities else None

# Enterprise value = market cap + debt - cash
ev = market_cap + total_debt - cash if market_cap and total_debt and cash is not None else None

# Revenue per share = total revenue divided by shares outstanding
revenue_per_share = revenue / shares if revenue and shares else None

# -----------------------------------------------
# Extract operating cash flow from cashflow statement (cf)
# -----------------------------------------------
cash_flow = None
for key in ["Cash Flowsfromusedin Operating Activities Direct", "Operating Cash Flow"]:
    if key in cf.index:
        cash_flow = cf.loc[key].iloc[0]
        break

# Cash flow per share = cash flow divided by shares
cash_flow_per_share = cash_flow / shares if cash_flow and shares else None

# -----------------------------------------------
# Average inventory (basic fallback using latest inventory)
# -----------------------------------------------
avg_inventory = inventory

# -----------------------------------------------
# Print all metrics by category
# -----------------------------------------------

print("Valuation Metrics:")
print("  P/E Ratio:", pe_ratio(price, eps))
print("  P/B Ratio:", pb_ratio(price, book_value))
print("  P/S Ratio:", ps_ratio(price, revenue_per_share))
print("  EV/EBITDA:", ev_ebitda(ev, ebitda))
print("  EV/Revenue:", ev_to_revenue(ev, revenue))

print("\nProfitability Metrics:")
print("  Gross Margin:", gross_margin(gross_profit, revenue))
print("  Operating Margin:", operating_margin(operating_income, revenue))
print("  Net Margin:", net_margin(net_income, revenue))
print("  ROA:", roa(net_income, total_assets))

print("\nEfficiency Metrics:")
print("  Asset Turnover:", asset_turnover(revenue, total_assets))
print("  Inventory Turnover:", inventory_turnover(fs.loc["Cost Of Revenue"].iloc[0] if "Cost Of Revenue" in fs.index else None, avg_inventory))

print("\nMarket Metrics:")
print("  Dividend Yield:", dividend_yield(dividend, price))
print("  Earnings Yield:", earnings_yield(eps, price))
print("  Price-to-Cash Flow:", price_to_cash_flow(price, cash_flow_per_share))



Valuation Metrics:
  P/E Ratio: 16.0
  P/B Ratio: 2.452483139178418
  P/S Ratio: 2.483761667231251
  EV/EBITDA: 13.076102847328132
  EV/Revenue: 2.8356184543657896

Profitability Metrics:
  Gross Margin: 0.39234230835301354
  Operating Margin: 0.11035645881931745
  Net Margin: 0.08841633259857083
  ROA: 0.05136164670629364

Efficiency Metrics:
  Asset Turnover: 0.580906775894976
  Inventory Turnover: 2.5855317986201243

Market Metrics:
  Dividend Yield: 0.02
  Earnings Yield: 0.0625
  Price-to-Cash Flow: 11.275791186084902
